In [96]:
import numpy as np
import pandas as pd
from glob import glob

In [97]:
core_videos = pd.read_csv('core_video_translation.csv')
driven_data = pd.read_csv('data/candSID_PanAfID_translations_old_platform_2020_12_18.txt', delimiter = "\t")
maureen_data = pd.read_csv('data/budongo_former_new_foldernames.csv')
bili_sample = pd.read_csv('data/bili_sample data.csv')

**Finding PanAf-500 metadata**

In [98]:
# Get list of PanAf-500 videos
driven_data.rename(columns={"clip.id":"id","video.id":"video", "new.video.id":"new_video"}, inplace=True)
core_videos = pd.read_csv('core_video_translation.csv')
panaf500 = [x.split('/')[-1].split('.')[0] for x in glob("/home/dl18206/Desktop/phd/data/panaf/obfuscated/videos/**/*.mp4", recursive=True)]

In [99]:
zooniverse_ids = core_videos[core_videos.obfu_id.isin(panaf500)].zooniverse_id.values

In [ ]:
df = driven_data[driven_data.id.isin(zooniverse_ids)]
df["filename"] = df.video.str.split('/').str[-1]
df["subdir"] = df.video.str.split('/').str[-2]
df["subdir_filename"] = df["subdir"] + str("/") + df["filename"]

df["new_filename"] = df.video.str.split('/').str[-1]
df["new_subdir"] = df.video.str.split('/').str[-2]
df["new_subdir_filename"] = df["new_subdir"] + str("/") + df["new_filename"]

df['same'] = np.where(df["subdir_filename"] == df["new_subdir_filename"], True, False)
df.drop(columns=['new_video', 'filename', 'subdir', 'new_filename', 'video', 'new_subdir', 'new_subdir_filename'], inplace=True)
df['subdir_filename'] = df['subdir_filename'].str.lower()

In [ ]:
# Match subdir_video in mdf
df['path'] = df.subdir_filename.str.split('/').str[0] + "_" + df.subdir_filename.str.split('/').str[1].str.split('.').str[0]

In [113]:
df.head()

,id,start.time,subdir_filename,same,path
466,ACP00058mb,0,nim_vid09_0546279_0820404_20140311/ek000033.avi,True,nim_vid09_0546279_0820404_20140311_ek000033
2137,ACP0004hk2,30,chimp/pict0077.avi,True,chimp_pict0077
3837,ACP0004v1t,0,nim_vid08_0549326_0825088_20130927/ek000102.avi,True,nim_vid08_0549326_0825088_20130927_ek000102
3997,ACP0004vat,0,nim_vid08_0549330_0825095_20140114/ek000040.avi,True,nim_vid08_0549330_0825095_20140114_ek000040
9317,ACP0004zqc,0,nim_vid22_0548118_0821891_20131120/ek000034.avi,True,nim_vid22_0548118_0821891_20131120_ek000034


In [129]:
# Extract country and site
df['site'] = df.path.str.split('_').str[0]
df.site.value_counts()

<ipython-input-129-2a92d2ee1a20>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['site'] = df.path.str.split('_').str[0]


bwi         203
bud          53
taieast      50
nim          39
tai          33
tai-e        25
lib          22
djo          22
kay          20
gbo          11
gangu         7
ngo           7
card 1.2      3
card 29       2
100ek113      1
chimp         1
cam 13        1
Name: site, dtype: int64

In [ ]:
df.path.unique()

In [125]:
df.path.str.split('/')

466         [nim_vid09_0546279_0820404_20140311_ek000033]
2137                                     [chimp_pict0077]
3837        [nim_vid08_0549326_0825088_20130927_ek000102]
3997        [nim_vid08_0549330_0825095_20140114_ek000040]
9317        [nim_vid22_0548118_0821891_20131120_ek000034]
                               ...                       
514776    [taieast_19_0690226_0643742_20150125+_ek000087]
515084     [taieast_39_0690932_0643624_20150204_pict0005]
519078    [taieast_29_0691019_0644568_20150212+_ek000006]
519214    [taieast_29_0691019_0644568_20150227+_ek000016]
521883    [taieast_67_0693033_0643496_20141218+_ek000042]
Name: path, Length: 500, dtype: object

In [107]:
df.path.unique().__len__()

423

In [101]:
csv_files = glob('/home/dl18206/Desktop/phd/data/panaf/maureen_annotations/data/sites/csv/**/*.csv', recursive=True)
sorted_csv_files = sorted(csv_files, key=lambda x: x.split('/')[-1])
initialiser, remainder = sorted_csv_files[0], sorted_csv_files[1:]
mdf = pd.read_csv(initialiser, encoding="ISO-8859-1")
for file in remainder:
    site = pd.read_csv(file, encoding="ISO-8859-1")
    mdf = pd.concat([mdf, site])
mdf['subdir_video'] = mdf.subfolder.astype(str) + '_' + mdf.video_file_name.astype(str)

In [109]:
mdf[mdf.subdir_video.isin(df.path)].research_site.value_counts()

east_nimba    106
budongo        72
grebo_sala     24
ngogo          13
Name: research_site, dtype: int64

In [131]:
mdf[mdf.subdir_video.isin(df.path)].habitat.value_counts()

forest - old secondary                66
forest - mixed, closed understorey    66
forest - mixed, open understorey      35
forest - lianas                       27
marshes                               10
gallery forest                         6
forest - marantaceae                   5
Name: habitat, dtype: int64

In [112]:
mdf[mdf.subdir_video.isin(df.path)].location_metadata.value_counts()

fruit tree                 87
trail                      68
trail / fruit tree         18
fruit tree / trail         15
feeding site / treefall     8
natural bridge              6
water source                5
buttress tree               3
nut cracking site           2
feeding site                2
treefall                    1
Name: location_metadata, dtype: int64

In [136]:
mdf[mdf.research_site=='budongo'].location_metadata.value_counts()

fruit tree        847
beehive            27
natural bridge     10
Name: location_metadata, dtype: int64

In [133]:
mdf.research_site.unique()

array(['bafing', 'bakoun', 'bateke', 'bili', 'boe', 'budongo', 'bwindi',
       'campo_maan', 'cnpe', 'cnpw', 'conkouati', 'dindefelo',
       'djouroutou', 'east_nimba', 'gashaka', 'geprenaf', 'gishwati',
       'goualougo', 'grebo_sala', 'kayan', 'korup', 'la_belgique',
       'loango', 'lope', 'mbe', 'mt_sangbe', 'mts_de_cristal', 'ngogo',
       'nyungwe', 'outamba', 'rubi_tele', 'sangaredi', 'sapo', 'sobeya',
       'sobory', 'taie', 'tair', 'ugalla_issa'], dtype=object)

In [111]:
mdf.columns

Index(['new_row_id', 'country', 'research_site', 'genus', 'species',
       'cam_coverage_area', 'location_metadata', 'habitat', 'utm_zone',
       'utm_long', 'utm_lat', 'cam_id', 'vid_res', 'vid_duration',
       'panaf_datasheet_comments', 'year', 'month', 'day', 'time_hr',
       'time_min', 'cell_id', 'subfolder', 'video_file_name', 'event_id',
       'min_number_chimps_per_video', 'max_number_chimps_per_video',
       'age_class', 'sex', 'tool_use', 'vocalization', 'bipedal',
       'camera_reaction', 'behavioral_context', 'other_species',
       'additional_comments', 'record_type', 'subdir_video'],
      dtype='object')

In [142]:
mdf['time_hr'].unique()

array([ 9., 17.,  6., 18., 16.,  8.,  7., 22., 19., 12., 14., 15., 10.,
       11., 13.,  5., 20.,  3.,  4., nan, 21.,  1.,  0.,  2., 23.])

In [34]:
maureen_data

,former_cell_id,former_foldername,former_videoname,new_cell_id,new_foldername,new_videoname,comments
0,Bud_A2,Bud_cam17_337202_192066_20120829,PICT0001.AVI,Bud_B3,Bud_cam17_337202_192066_20120829,PICT0001.AVI,EM: wrong cell_id
1,Bud_A2,Bud_cam17_337202_192066_20120829,PICT0002.AVI,Bud_B3,Bud_cam17_337202_192066_20120829,PICT0002.AVI,EM: wrong cell_id
2,Bud_A2,Bud_cam17_337202_192066_20120829,PICT0003.AVI,Bud_B3,Bud_cam17_337202_192066_20120829,PICT0003.AVI,EM: wrong cell_id
3,Bud_A2,Bud_cam17_337202_192066_20120829,PICT0004.AVI,Bud_B3,Bud_cam17_337202_192066_20120829,PICT0004.AVI,EM: wrong cell_id
4,Bud_A2,Bud_cam17_337202_192066_20120829,PICT0005.AVI,Bud_B3,Bud_cam17_337202_192066_20120829,PICT0005.AVI,EM: wrong cell_id
...,...,...,...,...,...,...,...
6759,Bud_C3,Bud_cam18_337159_190727_20121103,PICT0051.AVI,NaN,NaN,NaN,NaN
6760,Bud_C3,Bud_cam18_337159_190727_20121103,PICT0052.AVI,NaN,NaN,NaN,NaN
6761,Bud_C3,Bud_cam18_337159_190727_20121103,PICT0053.AVI,NaN,NaN,NaN,NaN
6762,Bud_C3,Bud_cam18_337159_190727_20121103,PICT0054.AVI,NaN,NaN,NaN,NaN


In [1]:
df = core_videos.join(driven_data.set_index('clip.id'), on='zooniverse_id')

NameError: name 'core_videos' is not defined

In [ ]:
df['video'] = df['video.id'].str.split('/').str[-1]
df['new_video'] = df['new.video.id'].str.split('/').str[-1]

In [ ]:
driven_data['video'] = driven_data['new.video.id'].str.split('/').str[-1]

In [ ]:
driven_data[driven_data.video=='PICT0026.AVI']

In [12]:
driven_data

,clip.id,start.time,video.id,new.video.id
0,ACP0001jwy,0,/Unchecked/Boe_D4/Boe_vid19_0620196_1318807_20...,boe can be found in new platform
1,ACP00013r1,0,/Unchecked/Boe_A7/Boe_vid07_0623144_1321628_20...,boe can be found in new platform
2,ACP0000rde,0,/checked/Boe_B6/Boe_vid34_0622100_1321086_2013...,boe can be found in new platform
3,ACP00017ol,0,/Unchecked/Boe_B3/Boe_vid10_0619171_1320672_20...,boe can be found in new platform
4,ACP0001erd,0,/Unchecked/Boe_B6/Boe_vid34_0622478_1321196_20...,boe can be found in new platform
...,...,...,...,...
522769,ACP000e3ef,15,TaiEast_MarieLyne/TaiEast_MarieLyne_CAMERA TRA...,Y:/primpanaf/22_Publications/03_MarieLyne Desp...
522770,ACP000e3en,15,TaiEast_MarieLyne/TaiEast_MarieLyne_CAMERA TRA...,Y:/primpanaf/22_Publications/03_MarieLyne Desp...
522771,ACP000e3dw,0,TaiEast_MarieLyne/TaiEast_MarieLyne_CAMERA TRA...,Y:/primpanaf/22_Publications/03_MarieLyne Desp...
522772,ACP000e3f7,0,TaiEast_MarieLyne/TaiEast_MarieLyne_CAMERA TRA...,Y:/primpanaf/22_Publications/03_MarieLyne Desp...


In [20]:
driven_data['subfolder'] = driven_data['video.id'].str.split('/').str[-2]
driven_data['video'] = driven_data['video.id'].str.split('/').str[-1]
driven_data['both'] = driven_data['subfolder'] + '/' + driven_data['video']

In [28]:
driven_data[driven_data['subfolder'].str.lower() == 'bili_vid11_243341_480435_20120830']

,clip.id,start.time,video.id,new.video.id,subfolder,video,both


In [30]:
driven_data[driven_data['subfolder'].str.startswith('bili')]

,clip.id,start.time,video.id,new.video.id,subfolder,video,both


In [39]:
## Species Assignment

In [23]:
core_videos = pd.read_csv('core_video_translation.csv')

In [24]:
core_videos

,obfu_id,zooniverse_id,filename,species,group__name,group_num,metadata__duration,metadata__start_time
0,GXB8ExhvZE,ACP000002l,ACP000002l.mp4,chimpanzee,quiet-wood-1,10,60.0,15.0
1,ZC0MKslbO1,ACP00000ep,ACP00000ep.mp4,chimpanzee,quiet-wood-1,10,60.0,0.0
2,gigV1Frh7Z,ACP00000g6,ACP00000g6.mp4,chimpanzee,quiet-wood-1,10,60.0,0.0
3,9SPJIFKuak,ACP00000ga,ACP00000ga.mp4,chimpanzee,quiet-wood-1,10,60.0,0.0
4,rAZ7FsTMce,ACP00000ma,ACP00000ma.mp4,chimpanzee,quiet-wood-1,10,60.0,0.0
...,...,...,...,...,...,...,...,...
5181,eobOSSMOzW,ACP000gnpq,ACP000gnpq.mp4,chimpanzee,aged-violet,0,15.0,0.0
5182,hp3BZkuAmb,ACP000gnpr,ACP000gnpr.mp4,chimpanzee,aged-violet,0,15.0,15.0
5183,HinLZooQYI,ACP000gnpt,ACP000gnpt.mp4,chimpanzee,aged-violet,0,15.0,30.0
5184,nYZg5nmdzG,ACP000gnpv,ACP000gnpv.mp4,chimpanzee,aged-violet,0,15.0,45.0


In [60]:
species_data = pd.read_csv('metadata.txt', delimiter = ",", header=None)

In [61]:
species_data # driven_data.set_index('clip.id'), on='zooniverse_id'

,0,1
0,009Bm4R6ke,Chimpanzee
1,00JRe5ZGg7,Chimpanzee
2,017u9bloM9,Chimpanzee
3,01qmAbY65k,Chimpanzee
4,02gegsFulz,Chimpanzee
...,...,...
5215,zweIo326Td,Chimpanzee
5216,zxacoc9ReX,Chimpanzee
5217,zy4K4YMj1X,Chimpanzee
5218,zzK1FJbIHv,Chimpanzee


In [ ]:
df = core_videos.join(species_data.set_index(0), on='obfu_id')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.rename({1: 'species'}, axis=1, inplace=True)
df.species = df.species.str.lower()

In [62]:
## Site assignment

In [78]:
core = pd.read_csv('core_video_translation.csv')
obfu_map = pd.read_csv('data/obfuscation_map_with_api_data.csv', dtype={'workflow_id': 'str'})

In [85]:
core

,obfu_id,zooniverse_id,filename,species
0,GXB8ExhvZE,ACP000002l,ACP000002l.mp4,chimpanzee
1,ZC0MKslbO1,ACP00000ep,ACP00000ep.mp4,chimpanzee
2,gigV1Frh7Z,ACP00000g6,ACP00000g6.mp4,chimpanzee
3,9SPJIFKuak,ACP00000ga,ACP00000ga.mp4,chimpanzee
4,rAZ7FsTMce,ACP00000ma,ACP00000ma.mp4,chimpanzee
...,...,...,...,...
5181,eobOSSMOzW,ACP000gnpq,ACP000gnpq.mp4,chimpanzee
5182,hp3BZkuAmb,ACP000gnpr,ACP000gnpr.mp4,chimpanzee
5183,HinLZooQYI,ACP000gnpt,ACP000gnpt.mp4,chimpanzee
5184,nYZg5nmdzG,ACP000gnpv,ACP000gnpv.mp4,chimpanzee


In [80]:
obfu_map.drop(columns=['group__zooniverse_id'], inplace=True)

In [81]:
join_df = core.merge(obfu_map, on=['obfu_id', 'zooniverse_id'], how='left')

In [82]:
join_df.columns

Index(['obfu_id', 'zooniverse_id', 'filename', 'species', 'idx', 'group__id',
       'group__name', 'location__standard__mp4', 'metadata__duration',
       'metadata__start_time', 'project_id', 'subject_id', 'workflow_id'],
      dtype='object')

In [83]:
join_df.drop(columns=['idx', 'group__id', 'project_id', 'workflow_id', 'subject_id', 'location__standard__mp4'], inplace=True)

In [98]:
join_df['group__name'] = join_df['group__name'].astype('category')

In [100]:
join_df['group'] = join_df['group__name'].cat.codes

In [104]:
join_df

,obfu_id,zooniverse_id,filename,species,group__name,metadata__duration,metadata__start_time,group
0,GXB8ExhvZE,ACP000002l,ACP000002l.mp4,chimpanzee,quiet-wood-1,60.0,15.0,10
1,ZC0MKslbO1,ACP00000ep,ACP00000ep.mp4,chimpanzee,quiet-wood-1,60.0,0.0,10
2,gigV1Frh7Z,ACP00000g6,ACP00000g6.mp4,chimpanzee,quiet-wood-1,60.0,0.0,10
3,9SPJIFKuak,ACP00000ga,ACP00000ga.mp4,chimpanzee,quiet-wood-1,60.0,0.0,10
4,rAZ7FsTMce,ACP00000ma,ACP00000ma.mp4,chimpanzee,quiet-wood-1,60.0,0.0,10
...,...,...,...,...,...,...,...,...
5181,eobOSSMOzW,ACP000gnpq,ACP000gnpq.mp4,chimpanzee,aged-violet,15.0,0.0,0
5182,hp3BZkuAmb,ACP000gnpr,ACP000gnpr.mp4,chimpanzee,aged-violet,15.0,15.0,0
5183,HinLZooQYI,ACP000gnpt,ACP000gnpt.mp4,chimpanzee,aged-violet,15.0,30.0,0
5184,nYZg5nmdzG,ACP000gnpv,ACP000gnpv.mp4,chimpanzee,aged-violet,15.0,45.0,0


In [110]:
column_to_move = join_df.pop("group")

# insert column with insert(location, column_name, column_value)

join_df.insert(5, "group_num", column_to_move)

In [111]:
join_df

,obfu_id,zooniverse_id,filename,species,group__name,group_num,metadata__duration,metadata__start_time
0,GXB8ExhvZE,ACP000002l,ACP000002l.mp4,chimpanzee,quiet-wood-1,10,60.0,15.0
1,ZC0MKslbO1,ACP00000ep,ACP00000ep.mp4,chimpanzee,quiet-wood-1,10,60.0,0.0
2,gigV1Frh7Z,ACP00000g6,ACP00000g6.mp4,chimpanzee,quiet-wood-1,10,60.0,0.0
3,9SPJIFKuak,ACP00000ga,ACP00000ga.mp4,chimpanzee,quiet-wood-1,10,60.0,0.0
4,rAZ7FsTMce,ACP00000ma,ACP00000ma.mp4,chimpanzee,quiet-wood-1,10,60.0,0.0
...,...,...,...,...,...,...,...,...
5181,eobOSSMOzW,ACP000gnpq,ACP000gnpq.mp4,chimpanzee,aged-violet,0,15.0,0.0
5182,hp3BZkuAmb,ACP000gnpr,ACP000gnpr.mp4,chimpanzee,aged-violet,0,15.0,15.0
5183,HinLZooQYI,ACP000gnpt,ACP000gnpt.mp4,chimpanzee,aged-violet,0,15.0,30.0
5184,nYZg5nmdzG,ACP000gnpv,ACP000gnpv.mp4,chimpanzee,aged-violet,0,15.0,45.0


In [114]:
join_df.to_csv('core_video_translation.csv', index=False)

In [1]:
import pandas as pd
from glob import glob

core = pd.read_csv('core_video_translation.csv')
core.obfu_id = core.obfu_id.str.lower()

p500 =  pd.DataFrame(dict(obfu_id=[x.split('/')[-1].split('.')[0].lower() for x in glob('/home/dl18206/Desktop/phd/data/panaf/PanAf500/videos/raw/all/**/*.mp4', recursive=True)]))

In [2]:
p500 = p500.merge(core, on='obfu_id')


In [3]:
p500

,obfu_id,zooniverse_id,filename,species,group__name,group_num,metadata__duration,metadata__start_time
0,aajn6zzbph,ACP000b3dy,ACP000b3dy.mp4,chimpanzee,cold-fire,1,15.0,0.0
1,dpw52gwsl1,ACP0005uy3,ACP0005uy3.mp4,chimpanzee,lingering-shape-9,8,15.0,30.0
2,jkf4v2wlfx,ACP000086p,ACP000086p.mp4,chimpanzee,quiet-wood-1,10,60.0,15.0
3,tsfcsskzig,ACP0003r9t,ACP0003r9t.mp4,chimpanzee,muddy-frost-7,9,15.0,30.0
4,d5d1ioy7ja,ACP00050c2,ACP00050c2.mp4,chimpanzee,crimson-dew-8,3,15.0,15.0
...,...,...,...,...,...,...,...,...
495,dvk4t8zjir,ACP0002ube,ACP0002ube.mp4,chimpanzee,red-water-6,11,15.0,0.0
496,ftshmgbzk4,ACP000bj97,ACP000bj97.mp4,chimpanzee,restless-star,12,15.0,0.0
497,0hu96jv2as,ACP000exsz,ACP000exsz.mp4,chimpanzee,aged-violet,0,15.0,30.0
498,hdj8vi3tjm,ACP00050aa,ACP00050aa.mp4,chimpanzee,crimson-dew-8,3,15.0,0.0


In [4]:
p500.to_csv('p500_video_translation.csv', index=False)

In [137]:
core[core.obfu_id=='GXB8ExhvZE']['species'].values[0]

'chimpanzee'

In [ ]:
!readlink -f 'core_video_translation.csv'